# Reading Stock Screeners based on Industry 

What we aim to do here is to get a list of all the companies and their technicals/fundamentals. 

## Steps followed
1. Import necessary libraries and setup chromedriver
2. Define a function that will get the Yahoo Finance
    1. Load the page
    2. Find the table in the page
    3. Extract the table
    4. Read the table as a dataframe 
3. Save the dataframe

In [34]:
import os

# Importing the necessary modules
import pandas as pd
import time
import warnings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

warnings.filterwarnings('ignore')

# Setting up the options for the webdriver
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)


In [35]:
def get_yahoo_table(yahoo_link, xpath):
    """Load a Yahoo Finance page and extract a table as a pandas DataFrame.

    :param yahoo_link: Yahoo Finance link to the page
    :param xpath: XPath of the table element

    :return: pandas DataFrame containing the table data
    """
    # Open a web browser to the provided Yahoo Finance link
    driver.get(yahoo_link)

    # Wait for the page to fully load (you might need to adjust the time)
    time.sleep(5)

    # Try to find the table on the page using the provided description (XPath)
    try:
        table_element = driver.find_element('xpath', xpath)
    except:
        # If the table is not found, show a message and return nothing
        print("Table element not found.")
        return None

    # Get the contents of the table in a format that computers can understand
    table_html = table_element.get_attribute('outerHTML')

    # Convert the table's contents into a structured format that can be easily worked with
    table_df = pd.read_html(table_html)[0]

    # Return the structured table as a result
    return table_df


def volume_string_to_int(str):
    multipliers = {'k':1000, 'M':1000000}
    if str[-1].isdigit():
        return int(str)
    mult = multipliers[str[-1]]
    return int(float(str[:-1])*mult)


In [36]:
# You will get this link by going to yahoo finance and doing the screener thing
yahoo_link = "https://finance.yahoo.com/screener/unsaved/6c46aa22-e5e2-42b1-a272-ab6fdd5a4aa3?offset=0&count=100"

# This should be the same for everyone but if you face issue drop a whatsapp message in the group
table_xpath = "/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/section/div/div[2]/div[1]"

table_data = get_yahoo_table(yahoo_link, table_xpath)


In [37]:
table_data

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AAPL,Apple Inc.,177.60,-0.37,-0.21%,13.662M,57.543M,2.776T,29.79,NaN
1,MSFT,Microsoft Corporation,325.49,-3.57,-1.08%,4.601M,24.9M,2.418T,33.66,NaN
2,NVDA,NVIDIA Corporation,435.01,-4.65,-1.06%,9.846M,49.922M,1.074T,105.32,NaN
3,TSM,Taiwan Semiconductor Manufacturing Company Lim...,88.34,-0.49,-0.55%,1.313M,8.921M,458.167B,15.53,NaN
4,AVGO,Broadcom Inc.,845.11,-4.89,-0.58%,260.404k,2.392M,348.845B,26.03,NaN
...,...,...,...,...,...,...,...,...,...,...
95,FJTSY,Fujitsu Limited,25.47,0.33,+1.31%,406903,91458,24.159B,18.06,NaN
96,HXGBF,Hexagon AB (publ),8.81,0.00,0.00%,148,4923,23.894B,20.99,NaN
97,KEYS,"Keysight Technologies, Inc.",131.75,-1.00,-0.75%,92839,1.349M,23.396B,20.98,NaN
98,HXGBY,Hexagon AB (publ),8.50,-0.06,-0.70%,1687,189100,22.986B,20.24,NaN


In [51]:
table_data.shape

table_data['Volume_Number'] = table_data['Volume'].apply(volume_string_to_int)
table_data.sort_values(by=['Volume_Number'], ascending=False)

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range,Volume_Number
0,AAPL,Apple Inc.,177.60,-0.37,-0.21%,13.662M,57.543M,2.776T,29.79,NaN,13662000
12,AMD,"Advanced Micro Devices, Inc.",101.76,-0.61,-0.60%,11.410M,61.521M,164.394B,NaN,NaN,11410000
71,PLTR,Palantir Technologies Inc.,15.07,-0.39,-2.49%,10.626M,63.544M,32.428B,NaN,NaN,10626000
2,NVDA,NVIDIA Corporation,435.01,-4.65,-1.06%,9.846M,49.922M,1.074T,105.32,NaN,9846000
27,ARM,Arm Holdings plc,54.94,-3.06,-5.28%,9.712M,79.856M,56.527B,144.97,NaN,9712000
...,...,...,...,...,...,...,...,...,...,...,...
96,HXGBF,Hexagon AB (publ),8.81,0.00,0.00%,148,4923,23.894B,20.99,NaN,148
57,IFNNF,Infineon Technologies AG,32.46,-1.54,-4.53%,118,1984,42.286B,12.88,NaN,118
22,SNEJF,Sony Group Corporation,88.73,0.00,0.00%,42,5709,109.429B,18.18,NaN,42
55,CNSWF,Constellation Software Inc.,2109.00,1.00,+0.05%,27,558,44.693B,91.74,NaN,27


In [52]:
if not os.path.exists("data/"):
    os.mkdir("data/")
    print("Data Directory Created!")

In [53]:
file_name = "utilities.csv"
table_data.to_csv("data/" + file_name)

In [54]:
table_data.columns

Index(['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change', 'Volume',
       'Avg Vol (3 month)', 'Market Cap', 'PE Ratio (TTM)', '52 Week Range',
       'Volume_Number'],
      dtype='object')